# ガウス型グラフィカルラッソによる異常検知

## 外れ値検出問題の場合

「異常検知と変化検知（講談社　井出　剛著）」
10.5.1 外れ値解析の場合　
より

異常度の算出式は以下

$\displaystyle a_i \equiv -\ln p ({x'}_i | \mathbf{x'}_{-i},\mathcal{D}) = \frac{1}{2}\ln \frac{2\pi}{\Lambda_{i,i}} + \frac{1}{2\Lambda_{i,i}}(\sum_{j=1}^M \Lambda_{i,j}{x'}_j)^2$


ホテリングT^2法に従い、異常度の算出する場合は前半部分を省略し、以下

$\displaystyle a_i \equiv \frac{1}{\Lambda_{i,i}}(\sum_{j=1}^M \Lambda_{i,j}{x'}_j)^2$


上記は各変数を個別に求める式であるが、各変数の異常度を一度に求める場合は、以下のような式になる。 

(正確には$\frac{1}{(\Lambda_{1,1},\Lambda_{2,2},\dots,\Lambda_{M,M})}$部分はベクトルなので、$\boldsymbol{X'}$のデータ数（行数）に合わせて、展開される必要がある。

よって展開はnumpyで自動的に行う）
 
 $\boldsymbol{a} \equiv \frac{1}{2} \ln \frac{2\pi}{(\Lambda_{1,1},\Lambda_{2,2},\dots,\Lambda_{M,M})} + \frac{1}{2(\Lambda_{1,1},\Lambda_{2,2},\dots,\Lambda_{M,M})}\odot(\boldsymbol{X'} \cdot \Lambda^T)^2$
 
ホテリングT^2法では以下となる。
  
$\boldsymbol{a} \equiv \frac{1}{(\Lambda_{1,1},\Lambda_{2,2},\dots,\Lambda_{M,M})}\odot(\boldsymbol{X'} \cdot \Lambda^T)^2$
  
よって行列計算を行う形式にする。

ガウス型グラフィカルラッソをScikit-LearnのGraphialLassoによって求め、そこから外れ値検出の異常度を求めるクラスを作成

In [52]:
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import GraphicalLasso

import numpy as np
from enum import Enum, auto

# 異常度の選択の列挙型
class OutlierDetectionAnomalyScoreType(Enum):
    NORMAL = auto() # 通常の算出
    HT2 = auto() # ホテリングT^2法

# ガウス型グラフィカルラッソによる異常検知クラス
class GaussianGraphicalLassoAnomalyDetection():
    # run_glを実行してから、get_anorm_scoresを実行する。
    
    # コンストラクタ
    # 訓練データを入れておく。
    def __init__(self, X_train):
        self.X_train = X_train
        
        # 標準化        
        self.std = StandardScaler()
        self.stdX_train = self.std.fit_transform(X_train)
        
        pass
    
    # 訓練データに対するグラフィカルラッソの実行(分散共分散行列、精度行列の取得)
    def run_gl(self, alpha=0.6, max_iter=50):
        # グラフィカルラッソの実行
        gl = GraphicalLasso(alpha=alpha, max_iter=max_iter).fit(self.stdX_train)
        
        # 算出した分散共分散行列と精度行列をインスタンス変数に格納
        self.cov = gl.covariance_
        self.prec = gl.precision_

        return self.cov, self.prec
    
    #　numpyデータに対しての外れ値検出の異常度の算出
    # X_testの列数とprecの列数は一致する必要がある。
    def get_outlier_detection_anomaly_scores(self, X_test, score_type=OutlierDetectionAnomalyScoreType.HT2):
        
        # テストデータを標準化（訓練データを元にする）
        stdX_test = self.std.transform(X_test)
        
        get_score_select = {
            # ホテリングT^2による異常度(第一項を無視し、２をかけた形式)
            # Sum部分は行列内積で代用
            OutlierDetectionAnomalyScoreType.HT2 :
                lambda X, prec : (1/np.diag(prec)) * (np.dot(np.nan_to_num(X), prec.T) ** 2),
            
            # 通常の異常度 (後半部分はホテリングT^2の計算を利用)
            OutlierDetectionAnomalyScoreType.NORMAL:
                lambda X, prec : (0.5 * np.log(2 * np.pi / np.diag(prec))) + (0.5 * get_score_select[OutlierDetectionAnomalyScoreType.HT2](X, prec))
        }
        self.outlier_detection_anomaly_scores = get_score_select[score_type](stdX_test, self.prec)
        return self.outlier_detection_anomaly_scores
    
    # 異常度が閾値を超えたかどうかをTrue,Falseで取得
    def islogical_outlier_detection_anomaly(self, a_th):
        return np.where(self.outlier_detection_anomaly_scores >= a_th, True, False)